In [73]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd

In [74]:
def read_story(url):
    
    response = requests.get(url)

    soup = BeautifulSoup(response.text, "html.parser")

    story = soup.find(id='dvAllLft')

    # remove the unwanted tags
    story.find('select', attrs = {'class':'inputArchive'}).decompose()
    picRt = story.find('div', attrs = {'class':'shadowInset picFrameRight'})
    if picRt: picRt.decompose()

    # remove all ads
    ads = story.findAll('div', attrs = {'class':'adSpaceEmbed'})
    for ad in ads: ad.decompose()

    picLt = story.find('div', attrs = {'class':'shadowInset picFrameLeft'})
    if picLt: picLt.decompose()

    left_story = story.find('td', attrs = {'align':'left'})
    if left_story: left_story.decompose()

    right_story = story.find('td', attrs = {'align':'right'})
    if right_story: right_story.decompose()

    story.find('div', attrs = {'id':'share'}).decompose()

    # story title
    title = soup.find('h1', attrs = {'class':'textAbnormalXLarge'}).text.upper()

    # moral
    moral = soup.find('h2', attrs = {'class':'textNormal bolden'}).text.strip()

    #print(title)
    #print(moral)

    # story
    content = ''
    for s in story.stripped_strings:
        #print(s)
        content += s + '\n'
        
    return title, moral, content, url

In [75]:
def gen_data():
    url = "http://www.talesofpanchatantra.com/short-stories-for-kids"
    response = requests.get(url)

    soup = BeautifulSoup(response.text, "html.parser")

    # Get all stories links
    all_links = soup.find('div', attrs={'id': 'listStories'})
    
    #urls = ["http://www.talesofpanchatantra.com/the-bug-and-the-poor-flea",
    #        'http://www.talesofpanchatantra.com/elephants-and-the-king-of-mice']
            

    for e in all_links.findAll('a'):
        u = e.get('href')
        yield read_story(u)
    
    #for u in urls:
    #    yield read_story(u)


In [76]:
df = pd.DataFrame(gen_data(), columns=['title', 'moral', 'content', 'url'])
df.to_csv(r"C:\Users\patilni\workdir\ML\units\story_teller\data\panchtantra.csv", index=False)

In [77]:
df.shape

(60, 4)

In [78]:
df.head()

,title,moral,content,url
0,THE MONKEY AND THE WEDGE,"""One, who interferes in other's work, surely c...",There was once a merchant who employed many ca...,http://www.talesofpanchatantra.com/the-monkey-...
1,THE JACKAL AND THE DRUM,"""Only the brave succeed in life"".","One day, a jackal called Gomaya was very hungr...",http://www.talesofpanchatantra.com/the-jackal-...
2,THE FALL AND RISE OF A MERCHANT,"""One should treat one and all, even the lowest...","In a city called Vardhamana, lived a very effi...",http://www.talesofpanchatantra.com/the-fall-an...
3,THE FOOLISH SAGE,"""Do not be taken in by the sweet words of a sw...","Once upon a time, there was a sage called Deva...",http://www.talesofpanchatantra.com/the-foolish...
4,FIGHTING GOATS AND THE JACKAL,"""Do not close your eyes to the impending dange...",One day while a sage was going through a jungl...,http://www.talesofpanchatantra.com/fighting-go...
